In [1]:
import pandas as pd
import json

file = "urdu_stories.json"
df = pd.read_json(file)

print(f"Length: {len(df)}")

Length: 500


removing author name (the first sentence)

In [2]:
def removeAuthor(text):
    if not text:
        return ""
    
    lines = text.splitlines()
    
    if len(lines) < 2:
        return text.strip()

    first_line = lines[0].strip()
    
    if len(first_line) < 60:
        cleaned_text = "\n".join(lines[1:])
        return cleaned_text.strip() # Remove leading whitespace
    else:
        # if not <60, likely not author name
        return text.strip()

df['content'] = df['content'].apply(removeAuthor)

removing any english characters if they exist

In [3]:
import re

def removeEnglish(text):
    if not text:
        return ""
    
    cleaned = re.sub(r'[a-zA-Z0-9]', '', text)
    
    return cleaned.strip()

df['content'] = df['content'].apply(removeEnglish)

standardising chars

In [4]:
import unicodedata

def standardizeUrdu(textData):
    if not textData:
        return ""
    
    textData = textData.replace("\\", '')

    # Unicode Normalization (NFC): to ensure that complex letters like
    # hamza + ya are stored using same number of bytes in the entire corpus
    normalizedText = unicodedata.normalize('NFC', textData)
    
    # punctuations
    normalizedText = normalizedText.replace('”', '"').replace('“', '"').replace('’', "'").replace('‘', "'")
    normalizedText = normalizedText.replace('.', '۔')
    
    # arabic chars
    replacements = {
        '\u0643': '\u06a9', # Kaf
        '\u064a': '\u06cc', # Ya
        '\u0647': '\u06c1', # He
    }
    
    for oldChar, newChar in replacements.items():
        normalizedText = normalizedText.replace(oldChar, newChar)
        
    return normalizedText.strip()

df['content'] = df['content'].apply(standardizeUrdu)

inserting "EOS", "EOP", "EOT" tokens

In [5]:
# Record Separator for EOS, Group Separator for EOP, and End of Text for EOT
eosToken = "\u241E" 
eopToken = "\u241D" 
eotToken = "\u0003"

def insertSpecialTokens(storyContent):
    if not storyContent:
        return ""
    
    # end of sentence
    storyContent = re.sub(r'([۔؟!])', r'\1 ' + eosToken, storyContent)
    
    # end of paragraph
    storyContent = re.sub(r'\n+', " " + eopToken + " ", storyContent)
    
    # end of story
    storyContent = storyContent.strip()
    if storyContent.endswith(eopToken):
        storyContent = storyContent[:-1].strip()
        
    storyContent += " " + eotToken
    
    return storyContent

df['content'] = df['content'].apply(insertSpecialTokens)

print(df['content'].iloc[0][:500])

میں ایک استاد ہوں اور میرا مضمون اسلامیات ہے۔ ␞نئے اسکول میں آج میرا پہلا دن تھا۔ ␞اسی وجہ سے خوشی بھی تھی اور ڈر بھی۔ ␞اسی خوشی میں، میں نے ناشتہ بھی برائے نام کیا اور وقت سے کچھ دیر پہلے ہی اسکول پہنچ گیا۔ ␞پرنسپل صاحب سے ملنے کے بعد مجھے ایک کلاس میں بھیج دیا گیا۔ ␞ ␝ کلاس روم میں خوب شور ہو رہا تھا۔ ␞تمام بچے اپنی عادت و فطرت کے مطابق زور زور سے باتیں کر رہے تھے۔ ␞میں نے کمرے میں قدم رکھا تو سب کو سانپ سونگھ گیا۔ ␞سب کے سب خاموشی سے سیدھے بیٹھ گئے۔ ␞پھر اچانک کلاس کی دائیں جانب سے "کلاس اسٹی


In [6]:
stories = df[['title', 'url', 'content']].to_dict(orient='records')

out = "urdu_stories_cleaned.json"
with open(out, 'w', encoding='utf-8') as f:
    json.dump(stories, f, ensure_ascii=False, indent=4)

In [7]:
# to verify
with open("urdu_stories_cleaned.json", "r", encoding="utf-8") as fileIn:
    loadedData = json.load(fileIn)

# Print the first story's content to check for backslashes
# Python will interpret the \" as a single " character
print(loadedData[0]['content'][:200])

میں ایک استاد ہوں اور میرا مضمون اسلامیات ہے۔ ␞نئے اسکول میں آج میرا پہلا دن تھا۔ ␞اسی وجہ سے خوشی بھی تھی اور ڈر بھی۔ ␞اسی خوشی میں، میں نے ناشتہ بھی برائے نام کیا اور وقت سے کچھ دیر پہلے ہی اسکول پہ
